# Schudeler Query from Silver to Gold for Items

Library

In [1]:
import pandas as pd
from pandas_gbq import read_gbq, to_gbq
import json

Project Info

In [2]:
project_id = "big-data-lol"
dataset_id_from = "Silver_LOL_Dataset"
dataset_id_to = "Gold_LOL_Dataset"
table_id_from_data = "silver_batch_items"
table_id_from_image = "silver_batch_img_item"
table_id_to = "gold_batch_item"

Lol current patch

In [3]:
current_patch = "14.12"

Query

In [5]:
query = f'''
SELECT REGEXP_EXTRACT(JSON_EXTRACT_SCALAR(data.image, '$.full'), r'(\d+)') AS number , img.image , data.name
FROM {project_id}.{dataset_id_from}.{table_id_from_data} as data join
{project_id}.{dataset_id_from}.{table_id_from_image} as img ON

REGEXP_EXTRACT(JSON_EXTRACT_SCALAR(data.image, '$.full'), r'(\d+)') = img.key

WHERE TIMESTAMP_TRUNC(data._PARTITIONTIME, DAY) = TIMESTAMP("2024-06-17")
and TIMESTAMP_TRUNC(img._PARTITIONTIME, DAY) = TIMESTAMP("2024-06-17")
'''

Query in BigQuery Dataset

In [6]:
df = read_gbq(query, project_id=project_id, dialect='standard')

Downloading: 100%|██████████|


In [7]:
df.head(5)

,number,image,name
0,223031,"""iVBORw0KGgoAAAANSUhEUgAAAEAAAABACAIAAAAlC+aJA...",Filo del Infinito
1,227031,"""iVBORw0KGgoAAAANSUhEUgAAAEAAAABACAIAAAAlC+aJA...",Filo de la Finitud
2,3031,"""iVBORw0KGgoAAAANSUhEUgAAAEAAAABACAIAAAAlC+aJA...",Filo del Infinito
3,7031,"""iVBORw0KGgoAAAANSUhEUgAAAEAAAABACAIAAAAlC+aJA...",Filo de la Finitud
4,447101,"""iVBORw0KGgoAAAANSUhEUgAAAIAAAACACAIAAABMXPacA...",Espada del Apostador


Schema create

In [8]:
from google.cloud import bigquery
schema = [

  bigquery.SchemaField('id', 'STRING', 'REQUIRED'),
  bigquery.SchemaField('img', 'STRING', 'REQUIRED'),
  bigquery.SchemaField('name', 'STRING', 'REQUIRED'),

]



In [9]:
client = bigquery.Client(project=project_id)

validated_path = '/tmp/validated_' + "champs_to_silver.csv"
# se transforma el df a csv
df.to_csv(validated_path, index=False)

print(validated_path)
# Cargar los datos validados en BigQuery
dataset_ref = client.dataset(dataset_id_to)
table_ref = dataset_ref.table(table_id_to)

#configuracion del job
job_config = bigquery.LoadJobConfig()

job_config.source_format = bigquery.SourceFormat.CSV
job_config.skip_leading_rows = 1
job_config.schema = schema
job_config.write_disposition = bigquery.WriteDisposition.WRITE_TRUNCATE


with open(validated_path, "rb") as source_file:
    job = client.load_table_from_file(source_file, table_ref, job_config=job_config)

job.result()


/tmp/validated_champs_to_silver.csv


LoadJob<project=big-data-lol, location=US, id=46d4fc7e-018f-45a1-ba48-86ba820a6bb0>

Job Configuration